In [1]:
# Ideally these should be set outside code.
import os
os.environ['PGHOST'] = "localhost"
os.environ['PGDATABASE'] = "crsp"

In [2]:
import os
import re
import json
import pandas as pd

from sqlalchemy import create_engine
from pandas.io.json import json_normalize

conn_string = 'postgresql://' + os.environ['PGHOST'] + '/' + os.environ['PGDATABASE']
engine = create_engine(conn_string)

target_schema = "se_features"

engine.execute("SET search_path TO %s, public" % target_schema)
rv = engine.execute("SELECT category FROM %s.liwc_2015" % target_schema)

categories = [ r['category'] for r in rv]

plan = """
    SELECT word_list
    FROM %s.liwc_2015 """ % target_schema + "WHERE category = %s"

mod_word_list = {}
for cat in categories:
    rows = list(engine.execute(plan, [cat]))
    word_list = rows[0]['word_list']
    mod_word_list[cat] = [re.sub('\*(?:\s*$)?', '[a-z]*', word.lower())
                            for word in word_list]

# Pre-compile regular expressions.
regex_list = {}
for key in mod_word_list.keys():
    regex = r"\b(?:" + "|".join(mod_word_list[key]) + r")(?=(?:[^a-zA-Z0-9_'-]|$))"
    regex_list[key] = re.compile(regex)
    
def liwc_counts(the_text):
    """Function to return number of matches against a LIWC category in a text"""
    # Construct a counter of the words and return as JSON
    text = re.sub(u'\u2019', "'", the_text).lower()
    the_dict = {cat: len(re.findall(regex_list[cat], text)) for cat in categories}
    return json.dumps(the_dict)

def expand_json(df, col):
    return pd.concat([df.drop([col], axis=1),
                      df[col].map(lambda x: json.loads(x)).apply(pd.Series)], axis=1)

In [3]:
regex_list['They']

re.compile(r"\b(?:their[a-z]*|them|themself|themselves|they|they'd|they'll|they've|theyd|theyll|theyve)(?=(?:[^a-zA-Z0-9_'-]|$))",
re.UNICODE)

In [4]:
df = pd.read_sql("""
    SELECT speaker_text 
    FROM streetevents.speaker_data 
    WHERE file_name = '1802211_T' AND context = 'pres'
        AND speaker_number = 3""", con = engine)

In [5]:
sample_text = """
    Okay. And then, as you look at the 132 franchise agreements signed year-to-date,
    it looks like about at least through this third quarter, slightly less 
    than half were net new agreements versus renewals or conversions. 
    The new guys coming in, what's the mix of brands they're choosing? 
    Are they -- where kind of in the scale are they kind of economy up?
            And what's the mix of brands they're coming over for?"""

In [6]:
sample_text = df['speaker_text'][0]

In [7]:
liwc_counts(sample_text)

'{"Function": 6286, "Pronoun": 1682, "Ppron": 1019, "I": 130, "We": 648, "You": 129, "SheHe": 9, "They": 103, "Ipron": 663, "Article": 898, "Prep": 1607, "Auxverb": 1186, "Power": 409, "Adverb": 600, "Conj": 729, "Negate": 138, "Verb": 2009, "Adj": 539, "Compare": 303, "Interrog": 156, "Number": 144, "Quant": 325, "Affect": 701, "Posemo": 519, "Negemo": 161, "Anx": 60, "Anger": 14, "Sad": 35, "Social": 1236, "Family": 0, "Friend": 4, "Female": 0, "Male": 11, "CogProc": 1357, "Insight": 241, "Cause": 266, "Discrep": 117, "Tentat": 374, "Certain": 151, "Differ": 326, "Percept": 117, "See": 72, "Hear": 21, "Feel": 10, "Bio": 51, "Body": 6, "Health": 36, "Sexual": 2, "Ingest": 11, "Drives": 1668, "Affiliation": 743, "Achieve": 332, "Reward": 238, "Risk": 119, "FocusPast": 469, "FocusPresent": 1346, "FocusFuture": 150, "Relativ": 1703, "Motion": 241, "Space": 981, "Time": 501, "Work": 851, "Leisure": 30, "Home": 42, "Money": 599, "Relig": 3, "Death": 3, "Informal": 76, "Swear": 0, "Netspeak